# ETL v4.1 #

In [1]:
import pandas as pd
import ast
from datetime import datetime

## movies_dataset.csv ##  
Desanidando los datos anidados

In [2]:
df = pd.read_csv('movies_dataset.csv', sep=',', encoding='utf-8', low_memory=False)

### Collection ###  
Obtener el nombre de 'Collection' en la columna 'btc_name'

In [3]:
df['belongs_to_collection'] = df['belongs_to_collection'].fillna('{}')
df['belongs_to_collection'] = df['belongs_to_collection'].apply(ast.literal_eval)
df_btc = pd.json_normalize(data=df['belongs_to_collection'])
df_btc = df_btc.add_prefix('btc_')
columnas_borrar = ['btc_id', 'btc_poster_path', 'btc_backdrop_path']
df_btc = df_btc.drop(columnas_borrar, axis=1)
df_btc = df_btc.fillna('noc')
df_btc.head(3)

,btc_name
0,Toy Story Collection
1,noc
2,Grumpy Old Men Collection


### Genres ###  
Obtener una lista de los generos 'genres' en la columna 'genres'

In [4]:
df['genres'] = df['genres'].fillna('{}')
df['genres'] = df['genres'].apply(ast.literal_eval)
df_genres = pd.json_normalize(data=df['genres'])
df_genres = df_genres.add_prefix('genre_')
df_genres = df_genres.apply(lambda x: x.str.get('name'))
df_genres['genres'] = df_genres.apply(lambda row: [value for value in row.values if value is not None], axis=1)
df_genres = df_genres['genres']
df_genres.head(3)

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
Name: genres, dtype: object

### Production Companies ###  
Obtener la lista de 'produccion_companies'en la columna 'companies'

In [5]:
df['production_companies'] = df['production_companies'].fillna('{}')
df['production_companies'] = df['production_companies'].apply(ast.literal_eval)
df_companies = pd.json_normalize(data=df['production_companies'])
df_companies = df_companies.add_prefix('companies_')
df_companies = df_companies.apply(lambda x: x.str.get('name'))
df_companies['companies'] = df_companies.apply(lambda row: [value for value in row.values if value is not None], axis=1)
df_companies = df_companies['companies']
df_companies.head(3)

0                            [Pixar Animation Studios]
1    [TriStar Pictures, Teitler Film, Interscope Co...
2                       [Warner Bros., Lancaster Gate]
Name: companies, dtype: object

### Production Countries ###  
Obtener la lista de 'produccion_countries'en la columna 'countries'

In [6]:
df['production_countries'] = df['production_countries'].fillna('{}')
df['production_countries'] = df['production_countries'].apply(ast.literal_eval)
df_countries = pd.json_normalize(data=df['production_countries'])
df_countries = df_countries.add_prefix('countries_')
df_countries = df_countries.apply(lambda x: x.str.get('name'))
df_countries['countries'] = df_countries.apply(lambda row: [value for value in row.values if value is not None], axis=1)
df_countries = df_countries['countries']
df_countries.head(3)

0    [United States of America]
1    [United States of America]
2    [United States of America]
Name: countries, dtype: object

### Spoken Languages ###  
Obtener la lista de 'spoken_languages'en la columna 'languages'

In [7]:
df['spoken_languages'] = df['spoken_languages'].fillna('{}')
df['spoken_languages'] = df['spoken_languages'].apply(ast.literal_eval)
df_languages = pd.json_normalize(data=df['spoken_languages'])
df_languages = df_languages.add_prefix('languages_')
df_languages = df_languages.apply(lambda x: x.str.get('name'))
df_languages['languages'] = df_languages.apply(lambda row: [value for value in row.values if value is not None], axis=1)
df_languages = df_languages['languages']
df_languages.head(3)

0              [English]
1    [English, Français]
2              [English]
Name: languages, dtype: object

### Borrar columnas innecesarias ###

In [8]:
df= df.dropna(subset='overview')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44512 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  44512 non-null  object 
 1   belongs_to_collection  44512 non-null  object 
 2   budget                 44512 non-null  object 
 3   genres                 44512 non-null  object 
 4   homepage               7765 non-null   object 
 5   id                     44512 non-null  object 
 6   imdb_id                44497 non-null  object 
 7   original_language      44502 non-null  object 
 8   original_title         44512 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             44507 non-null  object 
 11  poster_path            44166 non-null  object 
 12  production_companies   44512 non-null  object 
 13  production_countries   44512 non-null  object 
 14  release_date           44438 non-null  object 
 15  revenue

In [9]:
# columnas borrar del df original 'movies_dataset'
columnas_borrar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 
                   'belongs_to_collection', 'genres', 'production_companies', 
                   'production_countries', 'spoken_languages']
df = df.drop(columnas_borrar, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44512 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44512 non-null  object 
 1   id                 44512 non-null  object 
 2   original_language  44502 non-null  object 
 3   overview           44512 non-null  object 
 4   popularity         44507 non-null  object 
 5   release_date       44438 non-null  object 
 6   revenue            44506 non-null  float64
 7   runtime            44506 non-null  float64
 8   status             44441 non-null  object 
 9   tagline            20404 non-null  object 
 10  title              44506 non-null  object 
 11  vote_average       44506 non-null  float64
 12  vote_count         44506 non-null  float64
dtypes: float64(4), object(9)
memory usage: 4.8+ MB


### Crear df con nuevas columas de datos 'desanidados" ###
En 'df_movies' se incorporan las columnas de datos desanidados obtenidas anteriormente

In [10]:
# concatenado de todos los df
df_movies = pd.concat([df, df_btc, df_genres, df_companies, df_countries, df_languages], axis=1)

### Limpieza de datos y relleno de valores nulos ###  


In [11]:
# elimino las filas cuyo 'id' no es numérico
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies = df_movies.dropna(subset=['id'])
df_movies['id'] = df_movies['id'].astype('int64')

In [12]:
# rellenar valores nulos
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['budget'] = df_movies['budget'].fillna(0)
df_movies['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [13]:
# eliminar los valores nulos de 'release_date'
df_movies  = df_movies.dropna(subset='release_date')

### Crear la columna 'release_year' ###

In [14]:
def convertir_fecha(fecha_str):
    try:
        fecha = datetime.strptime(fecha_str, "%Y-%m-%d").date()
        return fecha
    except ValueError:
        return None

df_movies['release_date'] = df_movies['release_date'].apply(convertir_fecha)

In [15]:
def obtener_año(fecha):
    try:
        return fecha.year
    except AttributeError:
        return None
    
df_movies['release_year'] = df_movies['release_date'].apply(obtener_año)
df_movies['release_year'] = df_movies['release_year'].fillna(-1)
df_movies['release_year'] = df_movies['release_year'].astype(int)  # Convertir el año a entero

### Crear la columna retorno de inversión 'return' ###

In [16]:
# crear columna con retorno de inversion
def calcular_retorno(row):
    revenue = row['revenue']
    budget = row['budget']
    if revenue is not None and budget is not None and budget != 0:
        return revenue / budget
    else:
        return 0

df_movies['return'] = df_movies.apply(calcular_retorno, axis=1)

In [17]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44435 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44435 non-null  float64
 1   id                 44435 non-null  int64  
 2   original_language  44425 non-null  object 
 3   overview           44435 non-null  object 
 4   popularity         44435 non-null  object 
 5   release_date       44435 non-null  object 
 6   revenue            44435 non-null  float64
 7   runtime            44435 non-null  float64
 8   status             44371 non-null  object 
 9   tagline            20390 non-null  object 
 10  title              44435 non-null  object 
 11  vote_average       44435 non-null  float64
 12  vote_count         44435 non-null  float64
 13  btc_name           44435 non-null  object 
 14  genres             44435 non-null  object 
 15  companies          44435 non-null  object 
 16  countries          44435 no

In [18]:
df_movies.shape

(44435, 20)

In [19]:
df_movies.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,btc_name,genres,companies,countries,languages,release_year,return
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English],1995,12.451801
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,noc,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]",1995,4.043035
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]","[Warner Bros., Lancaster Gate]",[United States of America],[English],1995,0.000000
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,noc,"[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[United States of America],[English],1995,5.090760
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,[Comedy],"[Sandollar Productions, Touchstone Pictures]",[United States of America],[English],1995,0.000000


# credits.csv #

In [21]:
df_credits = pd.read_csv('credits.csv', sep=',', encoding='utf-8', low_memory=False)

In [22]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


### Cast ###  
Obtener la lista de 'cast' en la columna 'Cast'

In [23]:
df_credits['cast'] = df_credits['cast'].apply(ast.literal_eval)

def get_cast(cast_list):
    casts = [d['name'] for d in cast_list]
    return casts if casts else None

df_credits['Cast'] = df_credits['cast'].apply(get_cast)

### Directors ###  
Obtener los directores en la columna 'Directors'

In [24]:
df_credits['crew'] = df_credits['crew'].apply(ast.literal_eval)

# Crear una función para obtener una lista de nombres de directores si el trabajo es 'Director'
def get_directors(crew_list):
    directors = [d['name'] for d in crew_list if d['job'] == 'Director']
    return directors if directors else None

# Aplicar la función a la columna 'crew' y asignar el resultado a una nueva columna 'Directors'
df_credits['Directors'] = df_credits['crew'].apply(get_directors)
df_credits['Directors'] = df_credits['Directors'].fillna('[]')

In [25]:
# borrar columnas
columnas_borrar = ['cast', 'crew']
df_credits = df_credits.drop(columnas_borrar, axis=1)

In [26]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         45476 non-null  int64 
 1   Cast       43058 non-null  object
 2   Directors  45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [27]:
df_credits.shape

(45476, 3)

In [28]:
df_credits.head(3)

,id,Cast,Directors
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]


# Combinación de los datasets provistos #  
Utilizando 'merge' se combinan los datasets  
- movies_dataset.csv
- credits.csv

In [29]:
df_MoviesDataset = pd.merge(df_movies, df_credits, on='id')

In [30]:
df_MoviesDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44510 entries, 0 to 44509
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44510 non-null  float64
 1   id                 44510 non-null  int64  
 2   original_language  44500 non-null  object 
 3   overview           44510 non-null  object 
 4   popularity         44510 non-null  object 
 5   release_date       44510 non-null  object 
 6   revenue            44510 non-null  float64
 7   runtime            44510 non-null  float64
 8   status             44446 non-null  object 
 9   tagline            20417 non-null  object 
 10  title              44510 non-null  object 
 11  vote_average       44510 non-null  float64
 12  vote_count         44510 non-null  float64
 13  btc_name           44510 non-null  object 
 14  genres             44510 non-null  object 
 15  companies          44510 non-null  object 
 16  countries          445

In [31]:
df_MoviesDataset.shape

(44510, 22)

In [32]:
df_MoviesDataset.head(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,vote_count,btc_name,genres,companies,countries,languages,release_year,return,Cast,Directors
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English],1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter]
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,2413.0,noc,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]",1995,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston]
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,92.0,Grumpy Old Men Collection,"[Romance, Comedy]","[Warner Bros., Lancaster Gate]",[United States of America],[English],1995,0.000000,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch]


In [33]:
df_MoviesDataset.iloc[69]

budget                                                      50000000.0
id                                                               11859
original_language                                                   en
overview             Max Kirkpatrick is a cop who protects Kate McQ...
popularity                                                    5.912223
release_date                                                1995-11-03
revenue                                                     11534477.0
runtime                                                           91.0
status                                                        Released
tagline              He's a cop on the edge. She's a woman with a d...
title                                                        Fair Game
vote_average                                                       4.1
vote_count                                                        54.0
btc_name                                                           noc
genres

### Borrar columnas innecesarias ###

In [34]:
columnas_borrar2 = ['id', 'release_date', 'vote_average', 'status', 
                    'tagline', 'vote_count','languages', 'Cast']

df_MoviesDataset = df_MoviesDataset.drop(columnas_borrar2, axis=1)

In [35]:
df_MoviesDataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44510 entries, 0 to 44509
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44510 non-null  float64
 1   original_language  44500 non-null  object 
 2   overview           44510 non-null  object 
 3   popularity         44510 non-null  object 
 4   revenue            44510 non-null  float64
 5   runtime            44510 non-null  float64
 6   title              44510 non-null  object 
 7   btc_name           44510 non-null  object 
 8   genres             44510 non-null  object 
 9   companies          44510 non-null  object 
 10  countries          44510 non-null  object 
 11  release_year       44510 non-null  int32  
 12  return             44510 non-null  float64
 13  Directors          44510 non-null  object 
dtypes: float64(4), int32(1), object(9)
memory usage: 4.6+ MB


### Dataset pre-procesado ###
Guardar el dataset pre-procesado 'df_MoviesDataset' a un archivo csv para posterior uso

In [36]:
# Guardar DataFrame como archivo CSV
df_MoviesDataset.to_csv('MoviesDataset_v41.csv', index=False)